# Plotting the filters

In [1]:
import sys; sys.path.insert(0, "..")
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
# modelPath = ".model_checkpoint/validationset/bps-augmentednet-rmsproplr01-210526T143649/94-4.336-0.7938.hdf5" # best accuracy
# modelPath = ".model_checkpoint/prototyping/dropout-210528T215602/82-4.383-0.7917.hdf5" # with dropout
# modelPath = ".model_checkpoint/validationset/rmsprop-lr0001-200epochs-210529T090623/171-4.237-0.7809.hdf5" # low learning rate, 200 epochs
# modelPath = ".model_checkpoint/testset/haydnsun-augmentednet-alldata-210513T195252/23-3.821-0.8027.hdf5" # all data, evaluated on haydn
# modelPath = ".model_checkpoint/validationset/rmsprop-learningschedule-piecewise-210530T101943/55-4.175-0.7928.hdf5" # piecewise learning schedule
# modelPath = ".model_checkpoint/validationset/rmsprop-learningschedule-piecewise-stair-210530T170008/44-4.311-0.7949.hdf5" # piecewise stair
# modelPath = ".model_checkpoint/validationset/wirwtc-rotatinglrschedule-210530T215055/55-5.002-0.7453.hdf5" # wirwtc rotating lr
# modelPath = ".model_checkpoint/validationset/bps-lr01-0001-210530T233203/55-4.109-0.7956.hdf5" # bps, 01 to 0001
modelPath = ".model_checkpoint/validationset/bps-lr01-0001-210530T233203/46-4.088-0.7950.hdf5" # same as before, but lower loss instead of highest accuracy

In [4]:
filters, biases = model.layers[2].get_weights()

NameError: name 'model' is not defined

In [ ]:
filters = (filters - filters.min()) / (filters.max() - filters.min())

First filter of first convolutional layer

In [ ]:
sns.heatmap(filters[0], cmap="inferno")

# Exploring outputs of model

In [ ]:
model.summary()

In [ ]:
[l.name for l in model.layers]

In [ ]:
model.inputs

In [ ]:
newModel = keras.Model(inputs=model.inputs[0], outputs=model.layers[6].output)

In [ ]:
newModel.layers

In [ ]:
exampleInput = np.zeros((640, 19))
spellings = {
    (0, 10): "C--", (0, 11): "C-", (0, 0): "C", (0, 1): "C#", (0, 2): "C##",
    (1, 0): "D--", (1, 1): "D-", (1, 2): "D", (1, 3): "D#", (1, 4): "D##",
    (2, 2): "E--", (2, 3): "E-", (2, 4): "E", (2, 5): "E#", (2, 6): "E##",
    (3, 3): "F--", (3, 4): "F-", (3, 5): "F", (3, 6): "F#", (3, 7): "F##",
    (4, 5): "G--", (4, 6): "G-", (4, 7): "G", (4, 8): "G#", (4, 9): "G##",
    (5, 7): "A--", (5, 8): "A-", (5, 9): "A", (5, 10): "A#", (5, 11): "A##",
    (6, 9): "B--", (6, 10): "B-", (6, 11): "B", (6, 0): "B#", (6, 1): "B##",
}
numberOfSpellings = len(spellings)

for idx, (letter, pc) in enumerate(spellings.keys()):
    exampleInput[idx][letter] = 1
    exampleInput[idx][7 + pc] = 1

exampleInput = exampleInput.reshape((1, 640, 19))

In [ ]:
plt.figure(figsize=(20,10))
plt.pcolormesh(exampleInput[0][:numberOfSpellings+5].T, edgecolors="k")

In [ ]:
activations = newModel.predict(exampleInput)[0][:numberOfSpellings+5].T

In [ ]:
activations[0]

In [ ]:
activations.shape

In [ ]:
plt.figure(figsize=(20,10))
plt.pcolormesh(activations, edgecolors="k", cmap="inferno")

In [ ]:
for fidx, f in enumerate(activations):
    threshold = 0.5
    idxAct = [(idx, act) for idx, act in enumerate(f[:numberOfSpellings]) if act > threshold]
    idxAct.sort(key=lambda x: x[1], reverse=True)
    highActivations = [x[0] for x in idxAct]
#     print(highActivations)
    noteSpellings = list(spellings.values())
    notesActivated = [noteSpellings[idx] for idx in highActivations]
    print(f"Filter{fidx}: {notesActivated}")